In [2]:
'''
    This file takes care of all the data warehouse exchanges that would be made from the ML models 
    including and not limited to select, update, insert.
'''
%run ConfigFile.ipynb #import ConfigFile as cfg
import pyodbc
import pandas as pd
from azure.storage.blob import BlockBlobService


def fetch_data(query):
    import pyodbc
    import pandas as pd
    #Connection String from the config file
    cnxn = pyodbc.connect('DRIVER='+mysql['driver']+';SERVER='+mysql['server']+';PORT='+mysql['PORT']+';DATABASE='+mysql['database']+';UID='+mysql['username']+';PWD='+ mysql['password'])
    cursor = cnxn.cursor()
    
    #Stored procedure to fetch the call id and transcripts for the respective call id.
    data = pd.read_sql_query(query,cnxn)
    
    #Closing the connection
    cursor.close()
    cnxn.close()
    
    return data

def fetch_data_from_blob(top_level_container_name):
    import pyodbc
    import pandas as pd
    #Connection String from the config file
    block_blob_service = BlockBlobService(azure_details['azure_storage_name'], azure_details['azure_storage_account_key'])
    
    #Retrieve all the blob names from the container.
    blob_names= []
    generator = block_blob_service.list_blobs(top_level_container_name)
    for blob in generator:
        blob_names.append(blob.name)
        
    blob_file = block_blob_service.get_blob_to_text(top_level_container_name,blob_names[0]).content
    
    return blob_file